# Load dependencies

In [ ]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np

In [ ]:
gpus=tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

In [ ]:
cd D:/Course/DrivenData Project

# Import Files


In [ ]:
data_dir='Dataset'
t=os.path.join(data_dir,'train_features')
os.listdir(os.path.join(data_dir,'train_features'))

In [ ]:
df=pd.read_csv(os.path.join(data_dir,'train_features.csv'))
df

In [ ]:
train_labels = pd.read_csv(os.path.join(data_dir,'train_labels.csv'))
classes = train_labels.columns[1:]
train_labels = train_labels.iloc[:, 1:]
train_y = np.array([np.argmax(train_labels.iloc[i, :]) for i in range(train_labels.shape[0])])
l=list(train_y)

In [ ]:
img_filepath=df['filepath']
batch_size=32
img_filepath.shape

Map Image to its label

In [ ]:
from tensorflow import keras
from tensorflow.keras.utils import load_img,img_to_array,array_to_img,to_categorical
class MyTrainingData(keras.utils.Sequence):
    def __init__(self, file, labels, batchSize):
        self.file = file
        self.label = labels
        self.batchSize = batchSize
        self.n_batches = int(len(self.file) / self.batchSize)



    def on_epoch_end(self):  # it is called after every epoch
        self.file, self.label = shuffle(self.file, self.label)
        for i in range(50):
            print(self.file[i], self.label[i], 'file-label')


    def __len__(self):
        return self.n_batches
    

    # called after every batch to get new batch or new 32 images and labels
    def __getitem__(self, idx):
        # this method calls by fit method with idx ranging from 0 to len(training_exmaples) / batch_size =
        batchX = self.file[idx*self.batchSize: (idx+1)*self.batchSize]
        batchY = self.label[idx*self.batchSize: (idx+1)*self.batchSize]
        imgFiles = [tf.keras.utils.load_img(name, target_size=(224, 224, 3)) for name in batchX]   #loading 32 images
        imgFiles = [tf.keras.utils.img_to_array(img) for img in imgFiles]   #preprocessing
        imgFiles = [img / 255 for img in imgFiles] 
        batchY = to_categorical(batchY, 8)    # 4 represent number of classes (4 in that case)
        return np.array(imgFiles), np.array(batchY)


In [ ]:
cd D:/Course/DrivenData Project/Dataset

In [ ]:
data=MyTrainingData(img_filepath,l,batch_size)

In [ ]:
print(data.shape)
len(data)

# Model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, MaxPool2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.regularizers import l1, l2, l1_l2

In [ ]:
model=Sequential()
model.add(Conv2D(60,(3,3),input_shape=(224,224,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(60,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(100,activation='relu',activity_regularizer=l1(0.001)))
model.add(Dense(8))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='Adam' ,metrics=['accuracy'])

In [ ]:
model.fit(data,epochs=15,verbose=1)

Evaluation
